In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
df_data = pd.read_csv('data/dij_joint_civ.csv')

In [3]:
df_data.head()

,i,j,D_ij,civ_i,civ_j,civ_ij,comciv
0,AE,AL,-6.799441,islam,islam,islam,1
1,AE,AM,-2.050316,islam,ortho,NaN,0
2,AE,AO,-6.755312,islam,afric,NaN,0
3,AE,AR,-5.421352,islam,latam,NaN,0
4,AE,AS,-1.580247,islam,NaN,NaN,0


In [4]:
codes = pd.read_csv('data/codes.csv')

In [5]:
codes.head()

,Index,Country,Code
0,1,United Arab Emirates,AE
1,2,Angola,AO
2,3,Argentina,AR
3,4,Austria,AT
4,5,Australia,AU


In [9]:
edge_list = []
for index, row in df_data.iterrows():
    edge_list.append((row['i'], row['j'], {'cost':0, 'weight':row['D_ij']}))

In [10]:
graph = nx.Graph(edge_list)

In [11]:
graph